In [ ]:
using CSV, DataFrames, JuMP, Gurobi, Plots, HDF5, JLD

In [ ]:
# Some constants for the model

# R: hospitalization rates.
# 2x3 (age group x dose group)
# first row for ppl below 65 and second row for ppl above 65
# first to third column: 1, 2, and 0 doses of vaccine
R = Matrix([0.02 0.01 0.1; 0.05 0.01 0.25]) 
gamma = 0.1 # State can only vaccinate 10% of population per week

In [ ]:
data_folder = "real_world_data"
# Load csv files
case_file_g1 = data_folder * "/" * "case_data_g1.csv"
case_file_g2 = data_folder * "/" * "case_data_g2.csv"
population_file = data_folder * "/" * "population_data_age.csv"
vaccine_file = data_folder * "/" * "vaccine_data.csv"

case_data_g1 = CSV.read(case_file_g1, DataFrame)
case_data_g2 = CSV.read(case_file_g2, DataFrame)
population_data = CSV.read(population_file, DataFrame)
vaccine_data = CSV.read(vaccine_file, DataFrame)

P = Matrix(population_data)
CG1 = Matrix(case_data_g1)
CG2 = Matrix(case_data_g2)
V = Matrix(vaccine_data)
# convert population data to matrix

CG1 = CG1'
CG2 = CG2'
C = cat(CG1, CG2, dims=3)
P = P[:,2:3]
V = V[:,1]

println("C: ", size(C))
println("P: ", size(P))
println("V: ", size(V))

n_states = 1:size(case_data_g1, 2)
n_weeks = 1:size(case_data_g1, 1) 
n_weeks_2 = 1:size(case_data_g1, 1)-1 

n_groups = 1:size(P, 2)
n_doses = 1:3

total_states = length(n_states) 
total_weeks = length(n_weeks) 
println("Number of states: ", n_states)
println("Number of weeks: ", n_weeks)
println("Number of groups: ", n_groups)
println("Number of dose groups: ", n_doses)

In [ ]:
model = Model(Gurobi.Optimizer)

@variable(model, X[n_states, n_weeks, n_groups, n_doses] >= 0);
@variable(model, W[n_states, n_weeks, n_groups, n_doses] >= 0);
@variable(model, Y[n_states, n_weeks] >= 0);

@objective(model, Min, sum(sum(Y[i,j] for i in n_states) for j in n_weeks));

@constraint(model, hospitalizations[i in n_states, j in n_weeks],
       Y[i,j] == sum(sum(C[i,j,g] * W[i,j,g,d] * R[g,d] for g in n_groups) for d in n_doses))

@constraint(model, initial_vax_frac[i in n_states, g in n_groups, d in 1:2],
       W[i,1,g,d] == 0) 

@constraint(model, frac_bounds[i in n_states, j in n_weeks, g in n_groups, d in n_doses],
       W[i,j,g,d] <= 1)

@constraint(model, vax1_frac[i in n_states, j in n_weeks_2, g in n_groups],
       W[i,j+1,g,1] == W[i,j,g,1] + (X[i,j+1,g,1] - X[i,j+1,g,2]) / P[i,g]) 

@constraint(model, vax2_frac[i in n_states, j in n_weeks_2, g in n_groups],
       W[i,j+1,g,2] == W[i,j,g,2] + X[i,j+1,g,2] / P[i,g]) 

@constraint(model, novax_frac[i in n_states, j in n_weeks, g in n_groups],
       W[i,j,g,3] == 1 - W[i,j,g,1] - W[i,j,g,2])

@constraint(model, num_vax2_c1[i in n_states, j in 1:5, g in n_groups],
       X[i,j,g,2] == 0) 

@constraint(model, num_vax2_c2[i in n_states, j in 6:52, g in n_groups],
       X[i,j,g,2] <= sum(X[i,k,g,1] for k in 1:j-5) - sum(X[i,k,g,2] for k in 1:j-1)) 

@constraint(model, vax_supply[j in n_weeks],
       sum(sum(sum(X[i,j,g,d] for i in n_states) for g in n_groups) for d in n_doses) <= V[j]) 

@constraint(model, state_vax_capacity[i in n_states, j in n_weeks],
       sum(sum(X[i,j,g,d] for g in n_groups) for d in n_doses) <= gamma * (P[i,1] + P[i,2]));

In [ ]:
optimize!(model)
objective_value(model)

In [ ]:
W = value.(W)
X = value.(X)
Y = value.(Y);

# X_soln = zeros(total_states, total_weeks)
# Y_soln = zeros(total_states, total_weeks)
# W_soln = zeros(total_states, total_weeks)
# # Compute distances between all pairs of points
# for i in n_states
#     for j in n_weeks
#         X_soln[i,j] = X[i,j]
#         Y_soln[i,j] = Y[i,j]
#         W_soln[i,j] = W[i,j]
#     end
# end

# # save the solutions as a jld file
# save("solution_real.jld", "X_soln", X_soln, "Y_soln", Y_soln, "W_soln", W_soln)

In [ ]:
plot(sum(Matrix(Y), dims=2), xlabel="Week", ylabel="Hospitalizations", label="LO soln.")